# Diffusion-Shock Inpainting in $SE(2)$
Diffusion-shock inpainting (DS) is a technique to fill in missing structures in images, developed in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) and the follow-up paper ["Regularised Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://arxiv.org/abs/2309.08761). In this notebook, we will look at DS applied to images lifted into $SE(2)$.

In $\mathbb{R}^2$, we can describe DS in a PDE-based formulation as
$$
\partial_t u = g(\lvert \nabla (G_{\nu} * u) \rvert^2) \underbrace{\Delta u}_{\textrm{Diffusion}} - \left(1 - g(\lvert \nabla (G_{\nu} * u) \rvert^2)\right) \underbrace{\mathrm{sgn}(\partial_{\vec{w} \vec{w}} (G_{\sigma} * u)) \lvert \nabla u \rvert}_{\textrm{Shock}},
$$
in which $g: [0, \infty) \to (0, 1]$ is a decreasing function with $g(0) = 1$, $G_{\alpha}$ is a Gaussian with standard deviation $\alpha$, and $\vec{w}$ is the dominant eigenvector of the structure tensor. It is clear then that $g$ switches between applying diffusion and shock: if the gradient of the image is small, we mostly apply diffusion, but if the gradient is large, we mostly apply shock. This makes sense, since a large gradient implies that there is a feature there, which we would like to sharpen up. 

The signum in the shock term switches between erosion and dilation. If the second derivative with respect to the dominant eigenvector of the structure tensor is positive, then we perform erosion (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$); otherwise we perform dilation (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$). In regularised DS, the signum is replaced with a soft signum, so that the selection of erosion vs dilation is less sensitive to noise.

The signum of the second derivative of the dominant eigenvector of the structure tensor is not unlike the convexity criterion we know from studying vesselness; perhaps we could replace it?

What is the correct way to extend DS to $SE(2)$? It would make sense to keep the gradients and Laplacian. For the selection of erosion vs dilation we could again look at the vesselness convexity criterion. For switching between diffusion and shock, we could maybe use some sort of line/edge detector.

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False, device_memory_GB=3.5) #, kernel_profiler=True) # Use less than 4 so that we don't mix RAM and VRAM (?)
import numpy as np
import scipy as sp
# from PIL import Image
import matplotlib.pyplot as plt
# %matplotlib widget
import dsfilter

## Lines

In [ ]:
test_case = 2

In [ ]:
match test_case:
    case 1: # Grid of lines (black)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 0.
            u_ground_truth[(centre-2):(centre+3), :] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        # mask = (xs**2 + ys**2) < 0.6
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 255.

    case 2: # Grid of lines (alternating black and white)
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 4 # 7
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 255. - colour
            u_ground_truth[(centre-2):(centre+3), :] = colour
            colour = 255. - colour
        u_ground_truth = sp.ndimage.gaussian_filter(u_ground_truth, 1)

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        # mask = (xs**2 + ys**2) < 0.5
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 0.5 * 255.
    case 3: # Sine Wave
        dim_I, dim_J = 256, 256
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-2):(centre+3)] = 0.
            u_ground_truth[(centre-2):(centre+3), :] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        # mask = (xs**2 + ys**2) < 0.6
        l = 0.4
        mask = (xs**2 < l) * (ys**2 < l)

        u = u_ground_truth.copy()
        u[mask] = 255.


u = sp.ndimage.gaussian_filter(u, 1.) # Smooth for well-posed lifting.

clip = (u.min(), u.max())

mask = 1 - mask.astype(int)
mask = sp.ndimage.binary_erosion(mask, iterations=10, border_value=1).astype(int) # Deal with boundary artefacts.

dim_I, dim_J = u.shape
dim_K = 16 # 32
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Mask")
fig.colorbar(cbar, ax=ax[1]);

### Orientation Score

In [ ]:
cws_check = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/24)

In [ ]:
K = 1
print(θs[0, 0, K])
fig, ax, cbar = dsfilter.visualisations.plot_image_array(cws_check.real[K], x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title("$\psi$")
fig.colorbar(cbar, ax=ax);

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/24)
U = dsfilter.orientationscore.wavelet_transform(u, cws.real)
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ
Mask = np.transpose(np.array([mask for _ in range(dim_K)]), axes=(1, 2, 0)) # x, y, θ
U.shape, Mask.shape

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[2]);

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Reconstruction")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title("Reconstruction error")
fig.colorbar(cbar, ax=ax[2]);

### Preprocess Orientation Score
To remove artefacts in the orientation score caused by lifting masked data, we set the data within the (dilated) mask to a constant.

In [ ]:
U_preprocessed = dsfilter.SE2.utils.clean_mask_boundaries(U, Mask)

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(3, 3, figsize=(18, 15))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_preprocessed[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} preprocessed $U$")
fig.colorbar(cbar, ax=ax[1, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title(f"{K} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title(f"{K + 2} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4] * mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 2])
ax[2, 2].set_title(f"{K + 4} preprocessed $U$")
fig.colorbar(cbar, ax=ax[2, 2]);

### Run DS Inpainting

In [ ]:
# Works well for inpainting
# T_short = 0.2
# T_medium = 10.
# T_long = 50.
# T_mega_long = 500.

# ξ = 8 / dim_K

# G_D_inv = 1.8 * np.array((1., 0.1, 0.0))
# G_S_inv = np.array((1., 1., 0.0))
# # Internal regularisation for switching between dilation and erosion.
# σ_1, σ_2, σ_3 = np.array((2.5, 2.5, 0.6))
# # External regularisation for switching between dilation and erosion.
# ρ_1, ρ_2, ρ_3 = np.array((1., 1., 0.6))
# # Internal and external regularisation of gradient for switching between diffusion and shock.
# ν_1, ν_2, ν_3 = np.array((2.5, 2.5, 0.6))
# λ = 0.01 # Contrast parameter for switching between diffusion and shock.
# ε = 0.5 # Regularisation parameter for signum.

In [ ]:
T_short = 0.2
T_medium = 10.
T_long = 50.
T_mega_long = 500.

ξ = 8 / dim_K

G_D_inv = 1.8 * np.array((1., 0.1, 0.0))
G_S_inv = np.array((1., 1., 0.0))
# Internal regularisation for switching between dilation and erosion.
σ_1, σ_2, σ_3 = np.array((2.5, 2.5, 0.6))
# External regularisation for switching between dilation and erosion.
ρ_1, ρ_2, ρ_3 = np.array((1., 1., 0.6))
# Internal and external regularisation of gradient for switching between diffusion and shock.
ν_1, ν_2, ν_3 = np.array((2.5, 2.5, 0.6))
λ = 0.01 # Contrast parameter for switching between diffusion and shock.
ε = 0.5 # Regularisation parameter for signum.

In [ ]:
u_filtered_short, switch_DS_short, switch_morph_short = dsfilter.DS_filter_LI(U_preprocessed, Mask, θs, ξ, T_short, G_D_inv, G_S_inv, σ_1, σ_2, σ_3, ρ_1, ρ_2, ρ_3, ν_1, ν_2, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_medium, switch_DS_medium, switch_morph_medium = dsfilter.DS_filter_LI(U_preprocessed, Mask, θs, ξ, T_medium, G_D_inv, G_S_inv, σ_1, σ_2, σ_3, ρ_1, ρ_2, ρ_3, ν_1, ν_2, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_long, switch_DS_long, switch_morph_long = dsfilter.DS_filter_LI(U_preprocessed, Mask, θs, ξ, T_long, G_D_inv, G_S_inv, σ_1, σ_2, σ_3, ρ_1, ρ_2, ρ_3, ν_1, ν_2, ν_3, λ, ε=ε, dxy=dxy)
u_filtered_mega_long, switch_DS_mega_long, switch_morph_mega_long = dsfilter.DS_filter_LI(U_preprocessed, Mask, θs, ξ, T_mega_long, G_D_inv, G_S_inv, σ_1, σ_2, σ_3, ρ_1, ρ_2, ρ_3, ν_1, ν_2, ν_3, λ, ε=ε, dxy=dxy)

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array((u_filtered_long - u_filtered_mega_long)[..., 0], x_min, x_max, y_min, y_max, cmap="gray")
fig.colorbar(cbar, ax=ax);

In [ ]:
K = 1
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
u_change = u_filtered_mega_long - u_filtered_short
K = 0
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_change[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_change[..., K + 1], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_change[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_change[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
u_test = u_filtered_mega_long.copy()
u_test[mask > 0.] = 10.

In [ ]:
u_test[..., 0].max(), U[..., 0].max()

In [ ]:
K = 0
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K], x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[0])
ax[0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[1])
ax[1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", clip=clip, fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_short.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_medium.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_long.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(np.clip(u_filtered_mega_long.sum(-1), 0., 255.) - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 2
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_short[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_medium[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_mega_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
u_test = switch_morph_mega_long.copy()
u_test[mask > 0.] = 0.

In [ ]:
u_test[..., 0].max(), U[..., 0].max()

In [ ]:
K = 0
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K], x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_short.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_medium.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_long.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_mega_long.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
u_test = switch_DS_mega_long.copy()
u_test[mask > 0.] = 1.

In [ ]:
u_test[..., 0].max(), U[..., 0].max()

In [ ]:
K = 0
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K - 1], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K + 1], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_test[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 4
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short[..., K + 1], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short[..., K + 3], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_medium[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_mega_long[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_medium.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$T = {T_medium}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_long.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$T = {T_long}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_mega_long.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$T = {T_mega_long}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
# import h5py

In [ ]:
# filename = f".\\data\\u_short.hdf5"
# with h5py.File(filename, "w") as distance_file:
#     distance_file.create_dataset("Dataset1", data=u_filtered_short)
# filename = f".\\data\\u_medium.hdf5"
# with h5py.File(filename, "w") as distance_file:
#     distance_file.create_dataset("Dataset1", data=u_filtered_medium)
# filename = f".\\data\\u_long.hdf5"
# with h5py.File(filename, "w") as distance_file:
#     distance_file.create_dataset("Dataset1", data=u_filtered_long)
# filename = f".\\data\\u_mega_long.hdf5"
# with h5py.File(filename, "w") as distance_file:
#     distance_file.create_dataset("Dataset1", data=u_filtered_mega_long)

In [ ]:
# filename = f".\\data\\u_init.hdf5"
# with h5py.File(filename, "w") as distance_file:
#     distance_file.create_dataset("Dataset1", data=U)

## Planes

**TODO**

In [ ]:
# test_case = 1

In [ ]:
# match test_case:
#     case 1: # Figure 2 (top) BIG
#         dim_I, dim_J = 256, 256
#         u_ground_truth = np.ones((dim_I, dim_J)) * 255.
#         u_ground_truth[:25, :] = 0.
#         u_ground_truth[100:160, :] = 0.
#         u_ground_truth[-25:, :] = 0.

#         mask = np.ones((dim_I, dim_J))
#         mask[:, 100:160] = 0.
#         u = u_ground_truth.copy()
#         u[:, 100:160] = 0.5 * 255.

#         T_short = 1/3
#         T_medium = 15
#         T_long = 100
#         T_mega_long = 1000

#         σ = 3 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 3 # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.
#     case 2: # Figure 2 (top) small
#         dim_I, dim_J = 128, 128
#         u_ground_truth = np.ones((dim_I, dim_J)) * 255.
#         u_ground_truth[:12, :] = 0.
#         u_ground_truth[50:80, :] = 0.
#         u_ground_truth[-12:, :] = 0.

#         mask = np.ones((dim_I, dim_J))
#         mask[:, 50:80] = 0.
#         u = u_ground_truth.copy()
#         u[:, 50:80] = 0.5 * 255.

#         T_short = 1/3
#         T_medium = 15
#         T_long = 100
#         T_mega_long = 1000

#         σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 3 # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.
#     case 3: # Figure 3 (top)
#         dim_I, dim_J = 128, 128
#         u_ground_truth = np.ones((dim_I, dim_J)) * 0.5 * 255.
#         u_ground_truth[:65, :] = 0.
#         u_ground_truth[65:, :] = 1. * 255.

#         u = np.ones((dim_I, dim_J)) * 0.5 * 255.
#         u[64, 65] = 0.
#         u[65, 65] = 1. * 255.
#         mask = np.zeros((dim_I, dim_J))
#         mask[64:66, 65] = 1.

#         T_short = 1/3
#         T_medium = 100
#         T_long = 1000
#         T_mega_long = 10000

#         σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 1 # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.
#     case 4: # Figure 2 (bottom)
#         dim_I, dim_J = 256, 256
#         u_ground_truth = np.zeros((dim_I, dim_J))
#         u_ground_truth[:, 100:160] = 1. * 255.
#         u_ground_truth[100:160, :] = 1. * 255.

#         mask = np.ones((dim_I, dim_J))
#         mask[80:180, 80:180] = 0.
#         u = u_ground_truth.copy()
#         u[80:180, 80:180] = 0.5 * 255

#         T_short = 1/3
#         T_medium = 50
#         T_long = 250
#         T_mega_long = 2000

#         σ = 2 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 3 # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.
#     case 5: # Figure 8 (top) of RDS
#         dim_I, dim_J = 256, 256
#         xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
#         u_ground_truth = ((xs**2 + ys**2) < 0.8) * 255.

#         mask = np.ones((dim_I, dim_J))
#         mask[128:250, 5:128] = 0.
#         u = u_ground_truth.copy()
#         u[128:250, 5:128] = 0.5 * 255

#         T_short = 1/3
#         T_medium = 100
#         T_long = 1000
#         T_mega_long = 10000

#         σ = 2.5 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 2 # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.
#     case 6: # Figure 4
#         dim_I, dim_J = 256, 256
#         xs, ys = np.meshgrid(np.linspace(-1., 1., dim_I), np.linspace(-1., 1., dim_J), indexing="ij")
#         u_ground_truth = (ys > 0.700 * xs - 0.487) * (ys > -3.53 * xs - 1.77) * (ys < -0.502 * xs + 0.349) * 255.

#         radius = np.sqrt(0.0426)
#         mask = (((xs + 0.303)**2 + (ys + 0.699)**2 < radius**2) +
#                 ((xs - 0.695)**2 + ys**2 < radius**2) +
#                 ((xs + 0.699)**2 + (ys - 0.701)**2 < radius**2)) * 1.
#         u = np.random.random((dim_I, dim_J)) * 255. * (1. - mask) + u_ground_truth * mask

#         T_short = 1/3
#         T_medium = 100
#         T_long = 1000
#         T_mega_long = 10000

#         σ = 3.5 # Internal regularisation of structure tensor for switching between dilation and erosion.
#         ρ = 1.6 * σ # External regularisation of structure tensor for switching between dilation and erosion.
#         ν = 1.6 * σ # Internal regularisation of gradient for switching between diffusion and shock.
#         λ = 3. # Contrast parameter for switching between diffusion and shock.
#         ε = 0.15 * λ # Regularisation parameter for signum.

# dxy = 1.
# x_min, x_max = 0, u.shape[0] - 1
# y_min, y_max = 0, u.shape[1] - 1

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(12, 5))
# _, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
# ax[0].set_title("$u$")
# fig.colorbar(cbar, ax=ax[0])
# _, _, cbar = dsfilter.visualisations.plot_image_array(mask, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
# ax[1].set_title("Mask")
# fig.colorbar(cbar, ax=ax[1]);

In [ ]:
# u_filtered_short, switch_DS_short, switch_morph_short = dsfilter.DS_filter_R2(u, mask, T_short, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_medium, switch_DS_medium, switch_morph_medium = dsfilter.DS_filter_R2(u, mask, T_medium, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_long, switch_DS_long, switch_morph_long = dsfilter.DS_filter_R2(u, mask, T_long, σ, ρ, ν, λ, ε=ε, dxy=dxy)
# u_filtered_mega_long, switch_DS_mega_long, switch_morph_mega_long = dsfilter.DS_filter_R2(u, mask, T_mega_long, σ, ρ, ν, λ, ε=ε, dxy=dxy)

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
# ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
# ax[0, 1].set_title(f"$T = {T_medium}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
# ax[1, 0].set_title(f"$T = {T_long}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
# ax[1, 1].set_title(f"$T = {T_mega_long}$")
# fig.colorbar(cbar, ax=ax);

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_short - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
# ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_medium - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
# ax[0, 1].set_title(f"$T = {T_medium}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_long - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
# ax[1, 0].set_title(f"$T = {T_long}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(u_filtered_mega_long - u_ground_truth, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
# ax[1, 1].set_title(f"$T = {T_mega_long}$")
# fig.colorbar(cbar, ax=ax);

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
# ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
# ax[0, 1].set_title(f"$T = {T_medium}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
# ax[1, 0].set_title(f"$T = {T_long}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
# ax[1, 1].set_title(f"$T = {T_mega_long}$")
# fig.colorbar(cbar, ax=ax);

In [ ]:
# fig, ax = plt.subplots(2, 2, figsize=(12, 10))
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_short, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
# ax[0, 0].set_title(f"$T = {round(T_short, ndigits=2)}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_medium, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
# ax[0, 1].set_title(f"$T = {T_medium}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
# ax[1, 0].set_title(f"$T = {T_long}$")
# _, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_mega_long, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
# ax[1, 1].set_title(f"$T = {T_mega_long}$")
# fig.colorbar(cbar, ax=ax);

### Collapse Orientation Score

In [ ]:
def collapse_orientation_columns(U, σ=2, orientation_threshold=0.5):
    N_ors = U.shape[0]
    N_ors_new = N_ors//2
    U_unstacked = U[:N_ors_new] + U[N_ors_new:]
    U_mean = U_unstacked.mean()
    U_centred = U_unstacked - U_mean

    dθ = np.pi / (N_ors_new + 1)
    θs = np.arange(0, np.pi, dθ)
    A2_U = compute_grad_perp(U_centred, θs, σ)
    A22_U = compute_grad_perp(U_centred, θs, σ)
    orientationness = np.abs(A2_U) + np.abs(A22_U)
    k_opt = np.mod(orientationness.argmax(0), N_ors_new).astype(int)
    # k_opt = np.round(sp.ndimage.gaussian_filter(k_opt, σ)).astype(int)
    is_sufficiently_oriented = orientationness.max(0) >= orientation_threshold * orientationness.max()
    is_sufficiently_oriented = np.round(sp.ndimage.gaussian_filter(is_sufficiently_oriented * 1., σ)).astype(int)
    
    U_sharp = np.zeros_like(U_unstacked)
    U_projection_sum = U_centred.sum(0)
    U_projection_mean = U_centred.mean(0)
    for k in range(N_ors_new):
        U_sharp[k] = (
            (1 - is_sufficiently_oriented) * U_projection_mean +
            is_sufficiently_oriented * (k_opt == k) * U_projection_sum
        )
    return U_sharp + U_mean, k_opt, A2_U, is_sufficiently_oriented

def compute_grad_perp(U, θs, σ):
    A2_U = U.copy()
    for i, slice in enumerate(U):
        θ = θs[i]
        cos = np.cos(θ)
        sin = np.sin(θ)
        slice_x = sp.ndimage.gaussian_filter(slice, σ, order=(1, 0))
        slice_y = sp.ndimage.gaussian_filter(slice, σ, order=(0, 1))
        A2_U[i] = -sin * slice_x + cos * slice_y
    return A2_U

def compute_laplace_perp(U, θs, σ):
    A22_U = U.copy()
    for i, slice in enumerate(U):
        θ = θs[i]
        cos = np.cos(θ)
        sin = np.sin(θ)
        slice_xx = sp.ndimage.gaussian_filter(slice, σ, order=(2, 0))
        slice_xy = sp.ndimage.gaussian_filter(slice, σ, order=(1, 1))
        slice_yy = sp.ndimage.gaussian_filter(slice, σ, order=(0, 2))
        A22_U[i] = sin**2 * slice_xx - 2 * cos * sin * slice_xy + cos**2 * slice_yy
    return A22_U

In [ ]:
U_out, k_opt, A2_U, is_sufficiently_oriented = collapse_orientation_columns(U, σ=3, orientation_threshold=0.3)
U_out = np.transpose(U_out, axes=(1, 2, 0)) # x, y, θ

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(U_out[..., dim_K//4], x_min, x_max, y_min, y_max)
fig.colorbar(cbar, ax=ax);

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(is_sufficiently_oriented, x_min, x_max, y_min, y_max)
fig.colorbar(cbar, ax=ax);

### Old parameters

In [ ]:
# T_short = 1.
# T_medium = 10.
# T_long = 50.
# T_mega_long = 200.

# ξ = 8 / dim_K # 0.1
# # s_D = 100.
# # s_S = 100.
# # ζ_DS = 1.
# # s_DS = 100.
# # ζ_morph = 1.
# # s_morph = 100.
# # λ = 1. # Contrast parameter for switching between diffusion and shock.
# # ε = 0.15 * λ # Regularisation parameter for signum.

# # G_D_inv = np.array((1., 0., 0.))
# # G_S_inv = np.array((1., 0., 0.))
# # σ_s = 3 # Internal regularisation of structure tensor for switching between dilation and erosion.
# # σ_o = 0.01 # Internal regularisation of structure tensor for switching between dilation and erosion.
# # ρ_s = 1.6 * σ_s # External regularisation of structure tensor for switching between dilation and erosion.
# # ρ_o = 1.6 * σ_o # External regularisation of structure tensor for switching between dilation and erosion.
# # ν_s = 1.6 * σ_s # Internal regularisation of gradient for switching between diffusion and shock.
# # ν_o = 1.6 * σ_o # Internal regularisation of gradient for switching between diffusion and shock.

# G_D_inv = 0.2 * np.array((1., 1., 0.01))
# G_S_inv = np.array((1., 1., 0.01))
# σ_s = 1. # Internal regularisation of structure tensor for switching between dilation and erosion.
# σ_o = 0.1 # Internal regularisation of structure tensor for switching between dilation and erosion.
# ρ_s = 1. # 1.6 * σ_s # External regularisation of structure tensor for switching between dilation and erosion.
# ρ_o = 0.1 # 1.6 * σ_o # External regularisation of structure tensor for switching between dilation and erosion.
# ν_s = 1. # Internal regularisation of gradient for switching between diffusion and shock.
# ν_o = 0.5 # Internal regularisation of gradient for switching between diffusion and shock.
# λ = 0.25 # Contrast parameter for switching between diffusion and shock.
# ε = 3. # Regularisation parameter for signum.

In [ ]:
# print(1 / (np.sqrt(s_D) * ξ), 1 / (np.sqrt(s_D) * ξ), 1 / (np.sqrt(s_D) * dθ))
# print(1 / (np.sqrt(s_S) * ξ), 1 / (np.sqrt(s_S) * ξ), 1 / (np.sqrt(s_S) * dθ))
# print(1 / (np.sqrt(s_DS) * ξ), 1 / (np.sqrt(s_DS) * ξ * ζ_DS), 1 / (np.sqrt(s_DS) * ζ_DS * dθ))
# print(1 / (np.sqrt(s_morph) * ξ), 1 / (np.sqrt(s_morph) * ξ * ζ_morph), 1 / (np.sqrt(s_morph) * ζ_DS * dθ))

In [ ]:
# G_D_inv = 1 / (s_D * np.array((ξ**2, ξ**2, 10)))
# G_S_inv = 1 / (s_S * np.array((ξ**2, ξ**2, 10)))
# ν_s = 1 / (np.sqrt(s_DS) * ξ)
# ν_o = 1 / (np.sqrt(s_DS))
# σ_s = 1 / (np.sqrt(s_morph) * ξ)
# σ_o = 1 / (np.sqrt(s_morph))
# ρ_s = 1 / (np.sqrt(s_morph) * ξ)
# ρ_o = 1 / (np.sqrt(s_morph))
# print(G_D_inv, G_S_inv, ν_s, ν_o, σ_s, σ_o, ρ_s, ρ_o)

In [ ]:
# u_filtered_short, switch_DS_short, switch_morph_short = dsfilter.DS_filter_LI(U, Mask, θs, T_short, ξ, s_D, s_S, ζ_DS, s_DS, ζ_morph, s_morph, λ, ε=ε, dxy=dxy)
# u_filtered_medium, switch_DS_medium, switch_morph_medium = dsfilter.DS_filter_LI(U, Mask, θs, T_medium, ξ, s_D, s_S, ζ_DS, s_DS, ζ_morph, s_morph, λ, ε=ε, dxy=dxy)
# u_filtered_long, switch_DS_long, switch_morph_long = dsfilter.DS_filter_LI(U, Mask, θs, T_long, ξ, s_D, s_S, ζ_DS, s_DS, ζ_morph, s_morph, λ, ε=ε, dxy=dxy)
# u_filtered_mega_long, switch_DS_mega_long, switch_morph_mega_long = dsfilter.DS_filter_LI(U, Mask, θs, T_mega_long, ξ, s_D, s_S, ζ_DS, s_DS, ζ_morph, s_morph, λ, ε=ε, dxy=dxy)